In [2]:
import json 
import threading
import time

from joblib import load
import pandas as pd
from datetime import datetime

from flask import Flask, render_template, jsonify
from flask_cors import CORS

from api_scripts.binance import get_binance_prices 
from api_scripts.coin_gecko import get_coingecko_prices 
from api_scripts.coin_layer import get_coinlayer_prices, get_coinlayer_prices_yesterday 

In [1]:
def format_cgd(cgd):
    dataf = {}
    convert_name ={       
        "btc":"BTC",
        "ltc":"LTC",
        "ada":"ADA",
        "eth":"ETH",
        "usdt":"USDT",
    }
    for coin in cgd:
        dataf[convert_name[coin["Symbol"]]] = coin["Price"]
     
    return dataf
    
def format_bd(bd):
        return {"BTC":float(bd["BTCUSD"]),"LTC":float(bd["LTCUSD"]),"ADA":float(bd["ADAUSD"]),"ETH":float(bd["ETHUSD"]),"USDT":float(bd["USDTUSD"]),}

def opportunity(buy,price,sell,cost,coin):
    return{"coin":coin,
    "sellerMarket":buy,
    "buyPrice":price,
    "buyerMarket":sell,
    "sellPrice":cost }
    
def compare(m1,m2,coin):
    m1price = m1["data"][coin]
    m2price = m2["data"][coin]
    if(m1price != m2price): 
        if(m1price< m2price): 
            return opportunity(m1["Name"],m1price,m2["Name"],m2price,coin)
        else:
            return opportunity(m2["Name"],m2price,m1["Name"],m1price,coin)
    else: 
        return False
    
def generate_arbitrage_opportunities(stale):
    if stale == False:
        cgd = get_coingecko_prices()
        bd = get_binance_prices()
        clfd = get_coinlayer_prices()["rates"]
        # clfd = json.loads(get_coinlayer_prices())["rates"]
    else:
        with open("jsons/cgarb.json","r") as f:
            cgd = json.load(f)
        with open("jsons/barb.json","r") as f:
            bd = json.load(f)
        with open("jsons/clarb.json","r") as f:
            clfd = json.load(f)["rates"]

    clf = {"Name":"CoinLayer","data":clfd}
    cgf = {"Name":"CoinGecko","data":format_cgd(cgd)}
    bf = {"Name":"Binance","data":format_bd(bd)}

    opportunities = []
    for coin in clf["data"]:
        comps = [ compare(clf,cgf,coin),
             compare(clf,bf,coin),
             compare(cgf,bf,coin)]
            
        for comp in comps:
            if comp != 0:
                opportunities.append(comp)
    return opportunities

def format_yesterdays_data(stale):
    if stale == True:
        with open("jsons/clarbm2.json","r") as cl:
            yesterdata = json.load(cl)
    else:
        yesterdata = get_coinlayer_prices_yesterday()
        
    yd = datetime.strptime(yesterdata["date"], "%Y-%m-%d")
    yestercoins = yesterdata["rates"]
    formatted_data={}
    for coin in yestercoins:
        formatted_data[coin] = {
        "Open":[yestercoins[coin]["rate"]],
        "High":[yestercoins[coin]["high"]],
        "Low":[yestercoins[coin]["low"]],
        "Close":[yestercoins[coin]["rate"]],
        "Adj Close":[yestercoins[coin]["rate"]],
        "Volume":[yestercoins[coin]["vol"]],
        "year":[yd.year],
        "month":[yd.month],
        "day":[yd.day],}
    return formatted_data

def retrieveModels():
    coins = ['BTC','LTC','ETH','ADA','USDT']
    mls = {}
    for coin in coins:
        mls[coin] = load(f"ml/regressor_{coin}.joblib") 
    return mls

def model_preds():
    final_pred = {}
    coin_data = data2["yester"]
    for coin in coin_data:
        final_pred[coin] = list(models[coin].predict(pd.DataFrame(coin_data[coin])))[0]
    return final_pred

In [4]:
with open("jsons/cgarb.json","r") as cg:
    cgdd = json.load(cg)

# print(cgdd,type(cgdd))
fcgd = format_cgd(cgdd)
print(fcgd)

{'BTC': 26957, 'ETH': 1877.18, 'USDT': 1.0, 'ADA': 0.372, 'LTC': 94.98}


In [3]:
models = retrieveModels()   

data2 = {"data":generate_arbitrage_opportunities(True),"pred_data":format_yesterdays_data(True)}

In [4]:
app = Flask(__name__)

CORS(app,resources={r"/*": {"origins": ["https://turkishfilms.github.io"]}})

def get_data_on_delay():
    while True:
        data2["data"] = generate_arbitrage_opportunities(False)
        data2["yester"] = format_yesterdays_data(False)
        data2["pred_data"] = model_preds()
        time.sleep(600) 

@app.route('/')
def home():
   return render_template('index.html',data=data2["data"])

@app.route('/learn')
def a():
    return render_template('learn.html')

@app.route('/predictions')
def b():
    return render_template('predictions.html',data=json.dumps(data2["pred_data"]))

if(__name__ == "__main__"):
    t = threading.Thread(target=get_data_on_delay)
    t.start()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2023 21:58:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:00:31] "GET /learn HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:00:31] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2023 22:00:37] "GET /predictions HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:01:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:02:51] "GET /learn HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:02:52] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2023 22:04:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:52:03] "GET / HTTP/1.1" 200 -
